# Analyzing SAE Feature Activations from Parquet Files

This notebook loads and analyzes feature activations from parquet files that were generated by the `record_feature_activations.py` script. It processes the data in a memory-efficient way by loading files in batches.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass
import json
import gc
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Configuration

Set up the analysis parameters:

In [ ]:
@dataclass
class AnalysisConfig:
    """Configuration for feature analysis"""
    # Analysis parameters
    top_k_global: int = 50  # Top K frames globally for each feature
    top_s_per_episode: int = 3  # Top S frames per episode for each feature
    min_variance_threshold: float = 0.01  # Ignore low-variance features
    
    # Feature ranking criteria
    ranking_metric: str = 'variance'  # 'variance', 'range', 'sparsity', 'composite'
    sparsity_threshold: float = 0.1  # Threshold for considering a feature "active"
    
    # Data processing
    batch_size: int = 1000  # Number of episodes to process at once
    max_features_to_analyze: int = 100  # Max features to analyze in detail
    
    # Output settings
    output_dir: str = "./feature_analysis_results"
    create_plots: bool = True
    save_detailed_examples: bool = True

# Create config
config = AnalysisConfig()

## Feature Statistics Class

This class tracks statistics for each feature as we process the data:

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

## Process Parquet Files

Now let's process the parquet files in batches to compute feature statistics:

In [ ]:
def load_dataset_frame_analysis(dataset_id: int, output_dir: str) -> Dict:
    """Load frame analysis for a specific dataset"""
    frame_analysis_dir = Path(output_dir) / "frame_analysis"
    file_path = frame_analysis_dir / f"dataset_{dataset_id:03d}_frame_analysis.json"
    
    if not file_path.exists():
        raise FileNotFoundError(f"Frame analysis file not found: {file_path}")
    
    with open(file_path, 'r') as f:
        return json.load(f)

def get_most_active_features_in_episode(dataset_id: int, episode_id: int, output_dir: str, top_k: int = 10) -> List[int]:
    """Get the most frequently highly-active features in a specific episode"""
    data = load_dataset_frame_analysis(dataset_id, output_dir)
    
    if str(episode_id) not in data:
        return []
    
    episode_data = data[str(episode_id)]
    feature_counts = {}
    
    # Count how many times each feature appears as highly active
    for frame_idx, features in episode_data.items():
        for feature_info in features:
            feature_idx = feature_info['feature_idx']
            if feature_idx not in feature_counts:
                feature_counts[feature_idx] = 0
            feature_counts[feature_idx] += 1
    
    # Sort by count and return top_k
    sorted_features = sorted(feature_counts.items(), key=lambda x: x[1], reverse=True)
    return [feature_idx for feature_idx, count in sorted_features[:top_k]]

def get_frame_activation_summary(dataset_id: int, episode_id: int, frame_idx: int, output_dir: str) -> List[Dict]:
    """Get detailed activation summary for a specific frame"""
    data = load_dataset_frame_analysis(dataset_id, output_dir)
    
    if str(episode_id) not in data:
        return []
    
    episode_data = data[str(episode_id)]
    if str(frame_idx) not in episode_data:
        return []
    
    return episode_data[str(frame_idx)]

def analyze_feature_temporal_patterns(dataset_id: int, episode_id: int, feature_idx: int, output_dir: str) -> List[int]:
    """Get all frame indices where a specific feature was highly active in an episode"""
    data = load_dataset_frame_analysis(dataset_id, output_dir)
    
    if str(episode_id) not in data:
        return []
    
    episode_data = data[str(episode_id)]
    active_frames = []
    
    for frame_idx, features in episode_data.items():
        for feature_info in features:
            if feature_info['feature_idx'] == feature_idx:
                active_frames.append(int(frame_idx))
                break
    
    return sorted(active_frames)

# Example usage functions
def print_episode_summary(dataset_id: int, episode_id: int, output_dir: str):
    """Print a summary of feature activations for an episode"""
    print(f"Episode Summary - Dataset {dataset_id}, Episode {episode_id}")
    print("=" * 50)
    
    try:
        data = load_dataset_frame_analysis(dataset_id, output_dir)
        
        if str(episode_id) not in data:
            print("Episode not found in data")
            return
        
        episode_data = data[str(episode_id)]
        total_frames = len(episode_data)
        
        # Count unique features across all frames
        all_features = set()
        frame_counts = []
        
        for frame_idx, features in episode_data.items():
            frame_counts.append(len(features))
            for feature_info in features:
                all_features.add(feature_info['feature_idx'])
        
        print(f"Total frames: {total_frames}")
        print(f"Unique highly-active features: {len(all_features)}")
        print(f"Average highly-active features per frame: {np.mean(frame_counts):.1f}")
        print(f"Max highly-active features in a frame: {max(frame_counts) if frame_counts else 0}")
        
        # Most frequent features
        most_active = get_most_active_features_in_episode(dataset_id, episode_id, output_dir, top_k=5)
        print(f"\nMost frequently active features:")
        for i, feature_idx in enumerate(most_active):
            active_frames = analyze_feature_temporal_patterns(dataset_id, episode_id, feature_idx, output_dir)
            print(f"  {i+1}. Feature {feature_idx}: active in {len(active_frames)} frames")
    
    except Exception as e:
        print(f"Error: {e}")

# Example: uncomment to analyze a specific episode after running the frame analysis
# print_episode_summary(dataset_id=0, episode_id=0, output_dir=config.output_dir)


In [ ]:
def process_parquet_files(parquet_dir: str, config: AnalysisConfig) -> Tuple[Dict[int, FeatureStatistics], List[int]]:
    """Process parquet files in batches to compute feature statistics"""
    parquet_dir = Path(parquet_dir)
    parquet_files = sorted(list(parquet_dir.glob("*.parquet")))
    
    if not parquet_files:
        raise ValueError(f"No parquet files found in {parquet_dir}")
    
    logging.info(f"Found {len(parquet_files)} parquet files to process")
    
    # Initialize feature statistics
    feature_stats = {}
    
    # Process files in batches
    for i in tqdm(range(0, len(parquet_files), config.batch_size), desc="Processing batches"):
        batch_files = parquet_files[i:i + config.batch_size]
        
        # Process each file in the batch
        for file_path in tqdm(batch_files, desc="Processing files", leave=False):
            # Read parquet file
            df = pd.read_parquet(file_path)
            
            # Group by feature index
            for feature_idx, group in df.groupby('feature_index'):
                if feature_idx not in feature_stats:
                    feature_stats[feature_idx] = FeatureStatistics(feature_idx)
                
                # Add values to statistics
                feature_stats[feature_idx].add_values(
                    group['activation'].values,
                    group['frame_index'].tolist(),
                    group['episode_id'].tolist(),
                    group['dataset_id'].tolist()
                )
        
        # Force garbage collection after each batch
        import gc
        gc.collect()
    
    # Compute final statistics
    logging.info("Computing final statistics...")
    for stats in feature_stats.values():
        stats.compute_statistics(config.sparsity_threshold)
    
    # Rank features
    ranked_features = rank_features(feature_stats, config)
    
    return feature_stats, ranked_features

def rank_features(feature_stats: Dict[int, FeatureStatistics], config: AnalysisConfig) -> List[int]:
    """Rank features by importance"""
    # Filter features by variance threshold
    valid_features = [
        feature_idx for feature_idx, stats in feature_stats.items()
        if stats.variance >= config.min_variance_threshold
    ]
    
    # Rank by chosen metric
    feature_scores = [
        (feature_idx, feature_stats[feature_idx].get_ranking_score(config.ranking_metric))
        for feature_idx in valid_features
    ]
    
    # Sort by score (highest first)
    feature_scores.sort(key=lambda x: x[1], reverse=True)
    ranked_features = [feature_idx for feature_idx, score in feature_scores]
    
    # Update ranking information in statistics
    for rank, feature_idx in enumerate(ranked_features):
        feature_stats[feature_idx].variance_rank = rank
    
    logging.info(f"Ranked {len(ranked_features)} features by {config.ranking_metric}")
    return ranked_features

In [ ]:
def get_feature_batches(total_features: int, num_batches: int = 20) -> List[Tuple[int, int]]:
    """Split features into batches of roughly equal size"""
    batch_size = total_features // num_batches
    batches = []
    for i in range(num_batches):
        start = i * batch_size
        end = start + batch_size if i < num_batches - 1 else total_features
        batches.append((start, end))
    return batches

def process_feature_batch(parquet_dir: str, feature_batch: Tuple[int, int], config: AnalysisConfig) -> Tuple[Dict[int, FeatureStatistics], List[int]]:
    """Process a single batch of features"""
    parquet_dir = Path(parquet_dir)
    parquet_files = sorted(list(parquet_dir.glob("*.parquet")))
    
    if not parquet_files:
        raise ValueError(f"No parquet files found in {parquet_dir}")
    
    start_feature, end_feature = feature_batch
    logging.info(f"Processing features {start_feature} to {end_feature}")
    
    # Initialize feature statistics for this batch
    feature_stats = {}
    
    # Process files in batches
    for i in tqdm(range(0, len(parquet_files), config.batch_size), desc="Processing batches"):
        batch_files = parquet_files[i:i + config.batch_size]
        
        # Process each file in the batch
        for file_path in tqdm(batch_files, desc="Processing files", leave=False):
            # Read parquet file
            df = pd.read_parquet(file_path)
            
            # Filter for features in this batch
            df = df[df['feature_index'].between(start_feature, end_feature - 1)]
            
            # Group by feature index
            for feature_idx, group in df.groupby('feature_index'):
                if feature_idx not in feature_stats:
                    feature_stats[feature_idx] = FeatureStatistics(feature_idx)
                
                # Add values to statistics
                feature_stats[feature_idx].add_values(
                    group['activation'].values,
                    group['frame_index'].tolist(),
                    group['episode_id'].tolist(),
                    group['dataset_id'].tolist(),
                )
        
        # Force garbage collection after each batch
        import gc
        gc.collect()
    
    # Compute final statistics
    logging.info("Computing final statistics...")
    for stats in feature_stats.values():
        stats.compute_statistics(config.sparsity_threshold)
    
    # Rank features in this batch
    ranked_features = rank_features(feature_stats, config)
    
    return feature_stats, ranked_features

def find_top_examples_for_batch(parquet_dir: str, feature_batch: Tuple[int, int], 
                              ranked_features: List[int], config: AnalysisConfig) -> Dict[int, Dict[str, List]]:
    """Find top examples for a batch of features"""
    logging.info("Finding top examples from saved episode data...")
    
    parquet_dir = Path(parquet_dir)
    parquet_files = sorted(list(parquet_dir.glob("*.parquet")))
    
    start_feature, end_feature = feature_batch
    top_examples = {feature_idx: {'global_high': [], 'global_low': [], 'episode_examples': {}} 
                   for feature_idx in ranked_features[:config.max_features_to_analyze]}
    
    # Process files in batches
    for i in tqdm(range(0, len(parquet_files), config.batch_size), desc="Processing batches"):
        batch_files = parquet_files[i:i + config.batch_size]
        
        for file_path in tqdm(batch_files, desc="Processing files", leave=False):
            # Load episode data
            episode_df = pd.read_parquet(file_path)
            episode_id = episode_df['episode_id'].iloc[0] if len(episode_df) > 0 else None
            dataset_id = episode_df['dataset_id'].iloc[0] if len(episode_df) > 0 else None

            # Filter for features in this batch
            episode_df = episode_df[episode_df['feature_index'].between(start_feature, end_feature - 1)]
            
            # Process each feature
            for feature_idx in ranked_features[:config.max_features_to_analyze]:
                try:
                    feature_data = episode_df[episode_df['feature_index'] == feature_idx]
                    
                    if len(feature_data) == 0:
                        continue
                    
                    # Sort by activation value
                    feature_data = feature_data.sort_values('activation', ascending=False)
                    
                    # Update global top examples
                    for _, row in feature_data.head(config.top_k_global).iterrows():
                        example = {
                            'frame_idx': int(row['frame_index']),
                            'episode_idx': int(row['episode_id']),
                            'dataset_id': int(row['dataset_id']),
                            'activation_value': float(row['activation'])
                        }
                        
                        # Add to global high examples
                        if len(top_examples[feature_idx]['global_high']) < config.top_k_global:
                            top_examples[feature_idx]['global_high'].append(example)
                        else:
                            # Replace lowest if this is higher
                            min_example = min(top_examples[feature_idx]['global_high'], 
                                            key=lambda x: x['activation_value'])
                            if example['activation_value'] > min_example['activation_value']:
                                top_examples[feature_idx]['global_high'].remove(min_example)
                                top_examples[feature_idx]['global_high'].append(example)
                    
                    # Add per-episode examples
                    episode_top = feature_data.head(config.top_s_per_episode)
                    if dataset_id not in top_examples[feature_idx]['episode_examples']:
                        top_examples[feature_idx]['episode_examples'][dataset_id] = {}
                    if episode_id not in top_examples[feature_idx]['episode_examples'][dataset_id]:
                        top_examples[feature_idx]['episode_examples'][dataset_id][episode_id] = []
                    
                    for _, row in episode_top.iterrows():
                        example = {
                            'frame_idx': int(row['frame_index']),
                            'episode_idx': int(row['episode_id']),
                            'dataset_id': int(row['dataset_id']),
                            'activation_value': float(row['activation'])
                        }
                        top_examples[feature_idx]['episode_examples'][dataset_id][episode_id].append(example)
                except Exception as e:
                    logging.error(f"Error processing file {file_path}: {e}")
                    continue

        # Force garbage collection after each batch
        gc.collect()
    
    # Sort global examples
    for feature_idx in ranked_features[:config.max_features_to_analyze]:
        top_examples[feature_idx]['global_high'].sort(
            key=lambda x: x['activation_value'], reverse=True
        )
    
    return top_examples

def save_batch_results(feature_stats: Dict[int, FeatureStatistics],
                      ranked_features: List[int],
                      top_examples: Dict[int, Dict[str, List]],
                      batch_idx: int,
                      config: AnalysisConfig):
    """Save results for a single batch"""
    output_dir = Path(config.output_dir) / f"batch_{batch_idx:02d}"
    output_dir.mkdir(parents=True, exist_ok=True)

    # Save feature statistics
    stats_data = {
        str(feature_idx): stats.to_dict()
        for feature_idx, stats in feature_stats.items()
    }

    with open(output_dir / "feature_statistics.json", "w") as f:
        json.dump(stats_data, f, indent=2)

    # Save ranked features
    ranking_data = {
        'ranking_metric': config.ranking_metric,
        'total_features': len(feature_stats),
        'valid_features': len(ranked_features),
        'ranked_features': ranked_features[:config.max_features_to_analyze]
    }

    with open(output_dir / "feature_ranking.json", "w") as f:
        json.dump(ranking_data, f, indent=2)

    # Save top examples
    examples_dir = output_dir / "examples"
    examples_dir.mkdir(exist_ok=True)

    for feature_idx in ranked_features[:]:
        # Save global top examples
        if feature_idx in top_examples:
            feature_dir = examples_dir / f"feature_{feature_idx:03d}"
            feature_dir.mkdir(exist_ok=True)
            
            with open(feature_dir / "global_top_examples.json", "w") as f:
                json.dump(top_examples[feature_idx]['global_high'], f, indent=2)

            # Convert episode_examples keys to regular Python integers
            episode_examples = {}
            for dataset_id, dataset_examples in top_examples[feature_idx]['episode_examples'].items():
                episode_examples[int(dataset_id)] = {
                    int(episode_id): examples 
                    for episode_id, examples in dataset_examples.items()
                }

            # Save per-episode examples
            with open(feature_dir / "episode_top_examples.json", "w") as f:
                json.dump(episode_examples, f, indent=2)

    logging.info(f"Batch {batch_idx} results saved to {output_dir}")

def process_all_features(parquet_dir: str, total_features: int, config: AnalysisConfig):
    """Process all features in batches"""
    # Get feature batches
    feature_batches = get_feature_batches(total_features)
    
    # Process each batch
    for batch_idx, feature_batch in enumerate(feature_batches):
        logging.info(f"Processing batch {batch_idx + 1}/{len(feature_batches)}")
        
        # Process features in this batch
        feature_stats, ranked_features = process_feature_batch(parquet_dir, feature_batch, config)
        
        # Find top examples for this batch
        top_examples = find_top_examples_for_batch(parquet_dir, feature_batch, ranked_features, config)
        
        # Save results for this batch
        save_batch_results(feature_stats, ranked_features, top_examples, batch_idx, config)
        
        # Clear memory
        del feature_stats
        del ranked_features
        del top_examples
        import gc
        gc.collect()

def get_total_features_from_sample(parquet_dir: str) -> int:
    """Get the total number of features by reading a sample of parquet files"""
    parquet_dir = Path(parquet_dir)
    parquet_files = sorted(list(parquet_dir.glob("*.parquet")))
    
    if not parquet_files:
        raise ValueError(f"No parquet files found in {parquet_dir}")
    
    # Sample first few files to get feature range
    max_feature = 0
    for file_path in parquet_files[:10]:  # Check first 10 files
        df = pd.read_parquet(file_path)
        if len(df) > 0:
            max_feature = max(max_feature, df['feature_index'].max())
    
    logging.info(f"Detected maximum feature index: {max_feature}")
    return max_feature + 1  # Since features are 0-indexed


In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

## Run Analysis

Now let's run the analysis on your parquet files:

In [ ]:
# Set the path to your parquet files
parquet_dir = "../output/episode_activations"

# Auto-detect the total number of features from a sample of files
total_features = get_total_features_from_sample(parquet_dir)

# Process all features in batches
process_all_features(parquet_dir, total_features, config)

print("Batch processing complete! Results saved in separate batch directories.")
print(f"Check the output directory: {config.output_dir}")
print("Each batch contains:")
print("  - feature_statistics.json: Statistics for features in that batch")
print("  - feature_ranking.json: Ranked features for that batch")
print("  - examples/: Top examples for each feature in that batch")

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
class FeatureStatistics:
    """Statistics for a single feature"""
    
    def __init__(self, feature_idx: int):
        self.feature_idx = feature_idx
        self.values = []
        self.frame_indices = []
        self.episode_indices = []
        self.dataset_indices = []
        
        # Computed statistics
        self.mean = 0.0
        self.std = 0.0
        self.variance = 0.0
        self.min_val = 0.0
        self.max_val = 0.0
        self.range_val = 0.0
        self.range_75_95 = 0.0
        self.percentiles = {}
        self.sparsity = 0.0  # Fraction of values below threshold
        self.activity_rate = 0.0  # Fraction of values above threshold
        
        # Rankings
        self.variance_rank = 0
        self.range_rank = 0
        self.sparsity_rank = 0
        self.composite_rank = 0
    
    def add_values(self, values: np.ndarray, frame_indices: List[int], episode_indices: List[int], dataset_indices: List[int]):
        """Add new values to the statistics"""
        self.values.extend(values.tolist())
        self.frame_indices.extend(frame_indices)
        self.episode_indices.extend(episode_indices)
        self.dataset_indices.extend(dataset_indices)
    
    def compute_statistics(self, sparsity_threshold: float = 0.1):
        """Compute final statistics from collected values"""
        if not self.values:
            return
        
        values_array = np.array(self.values)
        
        self.mean = float(values_array.mean())
        self.std = float(values_array.std())
        self.variance = float(values_array.var())
        self.min_val = float(values_array.min())
        self.max_val = float(values_array.max())
        self.range_val = self.max_val - self.min_val
        
        # Percentiles
        percentile_points = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        self.percentiles = {
            p: float(np.percentile(values_array, p)) 
            for p in percentile_points
        }
        
        self.range_75_95 = self.percentiles[95] - self.percentiles[75]
        # Sparsity (fraction of values below threshold)
        self.sparsity = float((np.abs(values_array) < sparsity_threshold).mean())
        self.activity_rate = 1.0 - self.sparsity
    
    def get_ranking_score(self, metric: str) -> float:
        """Get score for ranking features"""
        if metric == 'variance':
            # return self.variance
            return self.variance
        elif metric == 'range':
            return self.range_val
        elif metric == 'sparsity':
            # Higher activity rate (lower sparsity) is better
            return self.activity_rate
        elif metric == 'composite':
            # Composite score: variance * range * activity_rate
            return self.variance * self.range_val * self.activity_rate
        else:
            raise ValueError(f"Unknown ranking metric: {metric}")
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for saving"""
        return {
            'feature_idx': self.feature_idx,
            'mean': self.mean,
            'std': self.std,
            'variance': self.variance,
            'min': self.min_val,
            'max': self.max_val,
            'range': self.range_val,
            'percentiles': self.percentiles,
            'sparsity': self.sparsity,
            'activity_rate': self.activity_rate,
            'num_samples': len(self.values),
            'rankings': {
                'variance_rank': self.variance_rank,
                'range_rank': self.range_rank,
                'sparsity_rank': self.sparsity_rank,
                'composite_rank': self.composite_rank
            }
        }

In [ ]:
def load_all_batch_results(output_dir: str) -> Tuple[Dict[int, FeatureStatistics], List[int]]:
    """Load and combine results from all batches"""
    output_dir = Path(output_dir)
    all_feature_stats = {}
    
    # Find all batch directories
    batch_dirs = sorted([d for d in output_dir.iterdir() if d.is_dir() and d.name.startswith('batch_')])
    
    for batch_dir in batch_dirs:
        stats_file = batch_dir / "feature_statistics.json"
        if stats_file.exists():
            with open(stats_file, 'r') as f:
                batch_stats = json.load(f)
            
            # Convert back to FeatureStatistics objects
            for feature_idx_str, stats_dict in batch_stats.items():
                feature_idx = int(feature_idx_str)
                feature_stats = FeatureStatistics(feature_idx)
                
                # Set the computed statistics
                feature_stats.mean = stats_dict['mean']
                feature_stats.std = stats_dict['std']
                feature_stats.variance = stats_dict['variance']
                feature_stats.min_val = stats_dict['min']
                feature_stats.max_val = stats_dict['max']
                feature_stats.range_val = stats_dict['range']
                feature_stats.percentiles = stats_dict['percentiles']
                feature_stats.sparsity = stats_dict['sparsity']
                feature_stats.activity_rate = stats_dict['activity_rate']
                
                all_feature_stats[feature_idx] = feature_stats
    
    # Rank all features together
    ranked_features = rank_features(all_feature_stats, config)
    
    return all_feature_stats, ranked_features

def get_top_features_across_batches(output_dir: str, top_k: int = 50) -> List[int]:
    """Get the top K features across all batches"""
    all_feature_stats, ranked_features = load_all_batch_results(output_dir)
    return ranked_features[:top_k]

def print_top_features(output_dir: str, top_k: int = 10):
    """Print information about top features"""
    all_feature_stats, ranked_features = load_all_batch_results(output_dir)
    
    print(f"Top {top_k} Features Across All Batches:")
    print("=" * 50)
    
    for i, feature_idx in enumerate(ranked_features[:top_k]):
        stats = all_feature_stats[feature_idx]
        print(f"{i+1:2d}. Feature {feature_idx:4d}:")
        print(f"    Range (75-95%): {stats.range_75_95:.4f}")
        print(f"    Variance:       {stats.variance:.4f}")
        print(f"    Range:          {stats.range_val:.3f}")
        print(f"    Activity Rate:  {stats.activity_rate:.3f}")
        print()



In [ ]:
def consolidate_batch_results(output_dir: str, top_k_features: int = 100):
    """Consolidate results from all batches into a single summary"""
    output_dir = Path(output_dir)
    
    # Load all results
    all_feature_stats, ranked_features = load_all_batch_results(output_dir)
    
    # Create consolidated directory
    consolidated_dir = output_dir / "consolidated"
    consolidated_dir.mkdir(exist_ok=True)
    
    # Save consolidated feature statistics for top features
    top_features = ranked_features[:top_k_features]
    consolidated_stats = {
        str(feature_idx): all_feature_stats[feature_idx].to_dict()
        for feature_idx in top_features
    }
    
    with open(consolidated_dir / "top_feature_statistics.json", "w") as f:
        json.dump(consolidated_stats, f, indent=2)
    
    # Save consolidated ranking
    ranking_data = {
        'ranking_metric': config.ranking_metric,
        'total_features_analyzed': len(all_feature_stats),
        'top_features_consolidated': top_k_features,
        'ranked_features': top_features
    }
    
    with open(consolidated_dir / "consolidated_ranking.json", "w") as f:
        json.dump(ranking_data, f, indent=2)
    
    # Collect top examples from batches for top features
    consolidated_examples = {}
    for feature_idx in top_features:
        # Find which batch this feature belongs to
        for batch_dir in sorted([d for d in output_dir.iterdir() if d.is_dir() and d.name.startswith('batch_')]):
            examples_file = batch_dir / "examples" / f"feature_{feature_idx:03d}" / "global_top_examples.json"
            if examples_file.exists():
                with open(examples_file, 'r') as f:
                    examples = json.load(f)
                consolidated_examples[feature_idx] = examples
                break
    
    # Save consolidated examples
    with open(consolidated_dir / "top_feature_examples.json", "w") as f:
        json.dump(consolidated_examples, f, indent=2)
    
    logging.info(f"Consolidated results saved to {consolidated_dir}")
    logging.info(f"Top {top_k_features} features consolidated")
    
    return top_features
